# Adding custom operations for model builder

This notebook describes ways to implement your own operations, and how to use them with model builder.

### Main chapters of this notebook:
1. Setup the environment
1. Add a custom operation for model builder
1. Prepare dataset and create dataloaders
1. Build model with custom operations
1. Check pretrain, search and tune phases

## Setup the environment

First, let's set up the environment and make some common imports.

In [ ]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# You may need to change this variable to match free GPU index
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from pathlib import Path

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_optimizer import RAdam

from enot.models import register_searchable_op
from enot.models import SearchSpaceModel
from enot.models.mobilenet import build_mobilenet
from enot.optimize import EnotPretrainOptimizer
from enot.optimize import EnotSearchOptimizer

from tutorial_utils.train_utils import accuracy
from tutorial_utils.train_utils import WarmupScheduler

from tutorial_utils.dataset import create_imagenette_dataloaders
from tutorial_utils.phases import tutorial_pretrain_loop
from tutorial_utils.phases import tutorial_search_loop
from tutorial_utils.phases import tutorial_train_loop

## Add custom operation for model builder

To create custom operation, and use it with our model builders (`build_mobilenet`), follow these rules:
1. register operation in our framework using `@register_searchable_op(name)`;
1. it should accept 4 positional arguments: `in_channels`, `out_channels`, `stride`, `use_skip_connection`. All other arguments should have default value, or must be added in short config (see the next paragraph);
1. if you want to use the short config format, you must set parameter parsing rules;
1. if you need to search architecture w.r.t its latency, you must add `LatencyMixin` to you custom operation (see "Tutorial - using latency optimization").

#### Example of parameter parsing rules:

If you want to write `MyOp_k=3_a=relu` instead of `{'op_type': 'MyOp', 'kernel_size': 3, 'activation': 'relu'}`, you need the following rules:
```
{
  'k': ('kernel_size', int),
  'a': ('activation', str),
}
```

In [ ]:
activations = {
    'relu': nn.ReLU,
    'sigmoid': nn.Sigmoid,
}

# defineing short parameter parsing rules
# format: {short_param_name: (original_param_name, parser)}
short_args = {
    'k': ('kernel_size', int),
    'a': ('activation', lambda x: activations[x])
}


@register_searchable_op('MyOp', short_args)
class MyOperation(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        stride,
        kernel_size=3,
        activation=nn.ReLU,
        padding=None,   
        use_skip_connection=True,
    ):
        super().__init__()

        self.use_skip_connection = use_skip_connection 

        if padding is None:
            padding = (kernel_size - 1) // 2

        self.operation = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=kernel_size, 
                stride=stride, 
                padding=padding,
            ),
            activation(),  # for example, you can use activation before batch normalization layer
            nn.BatchNorm2d(out_channels),
        )

    def forward(self, x):
        y = self.operation(x)

        if self.use_skip_connection and x.shape == y.shape:
            y += x

        return y

## Prepare dataset and create dataloaders

In [ ]:
ENOT_HOME_DIR = Path.home() / '.enot'
ENOT_DATASETS_DIR = ENOT_HOME_DIR / 'datasets'
PROJECT_DIR = ENOT_HOME_DIR / 'custom_operation'

ENOT_HOME_DIR.mkdir(exist_ok=True)
ENOT_DATASETS_DIR.mkdir(exist_ok=True)
PROJECT_DIR.mkdir(exist_ok=True)

dataloaders = create_imagenette_dataloaders(
    dataset_root_dir=ENOT_DATASETS_DIR, 
    project_dir=PROJECT_DIR,
    input_size=(224, 224),
    batch_size=32,
    imagenette_kind='imagenette2-320',
)

## Build model with custom operations

In [ ]:
SEARCH_OPS = [
    'MIB_k=3_t=6',
    'MyOp_k=3',  # Notice that you can omit parameters with default values
    'MyOp_k=3_a=sigmoid',
]

# build model
model = build_mobilenet(
    search_ops=SEARCH_OPS,
    num_classes=10,
    blocks_out_channels=[24, 32, 64, 96, 160, 320],
    blocks_count=[2, 2, 2, 1, 2, 1],
    blocks_stride=[2, 2, 2, 1, 2, 1],
)

# move model to search space
search_space = SearchSpaceModel(model).cuda()

## Check pretrain search and tune phases

Let's check that everything works.

In [ ]:
N_EPOCHS = 3
N_WARMUP_EPOCHS = 1
len_train = len(dataloaders['pretrain_train_dataloader'])

optimizer = SGD(params=search_space.model_parameters(), lr=0.06, momentum=0.9, weight_decay=1e-4)
enot_optimizer = EnotPretrainOptimizer(search_space=search_space, optimizer=optimizer)
scheduler = CosineAnnealingLR(optimizer, T_max=len_train*N_EPOCHS, eta_min=1e-8)
scheduler = WarmupScheduler(scheduler, warmup_steps=len_train*N_WARMUP_EPOCHS)
loss_function = nn.CrossEntropyLoss().cuda()

tutorial_pretrain_loop(
    epochs=N_EPOCHS,
    search_space=search_space,
    enot_optimizer=enot_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['pretrain_train_dataloader'],
    validation_loader=dataloaders['pretrain_validation_dataloader'],
    scheduler=scheduler,
)

In [ ]:
optimizer = RAdam(search_space.architecture_parameters(), lr=0.01)
enot_optimizer = EnotSearchOptimizer(search_space=search_space, optimizer=optimizer)

tutorial_search_loop(
    epochs=5,
    search_space=search_space,
    enot_optimizer=enot_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['search_train_dataloader'],
    validation_loader=dataloaders['search_validation_dataloader'],
    latency_loss_weight=0,
    latency_type=None,
    scheduler=None,
)

In [ ]:
# get regular model with the best architecture
best_model = search_space.get_network_with_best_arch().cuda()

In [ ]:
optimizer = RAdam(best_model.parameters(), lr=5e-3, weight_decay=4e-5)

tutorial_train_loop(
    epochs=5,
    model=best_model,
    optimizer=optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['tune_train_dataloader'],
    validation_loader=dataloaders['tune_validation_dataloader'],
    scheduler=None,
)